In [2]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq 

llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model="llama3-70b-8192",
    temperature=0,
)

response = llm.invoke("What is the meaning of culture?")
print(response.content)

What a profound and complex question!

Culture is a multifaceted concept that has been debated and defined by scholars, anthropologists, sociologists, and philosophers across various disciplines. While there is no single, universally accepted definition, here's a comprehensive overview:

**Culture** refers to the collective values, beliefs, norms, customs, practices, and traditions that are shared by a group of people, often passed down from one generation to the next. It encompasses the ways in which people think, feel, behave, and interact with each other and their environment.

Culture is shaped by a combination of factors, including:

1. **History**: The past experiences, events, and traditions that have influenced a group's values and practices.
2. **Social structure**: The organization of society, including social institutions, roles, and relationships.
3. **Language**: The system of communication, including spoken and written language, symbols, and signs.
4. **Environment**: The

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://job-boards.greenhouse.io/planetlabs/jobs/6814646?gh_src=e56b92851us")
docs = loader.load()
page_data = docs[0].page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Job Application for Working Student, Image Quality Support at PlanetBack to jobsNewWorking Student, Image Quality SupportBerlin, GermanyApplyWelcome to Planet. We believe in using space to help life on Earth.
Planet designs, builds, and operates the largest constellation of imaging satellites in history. This constellation delivers an unprecedented dataset of empirical information via a revolutionary cloud-based platform to authoritative figures in commercial, environmental, and humanitarian sectors. We are both a space company and data company all rolled into one.
Customers and users across the globe use Planet's data to develop new technologies, drive revenue, power research, and solve our world’s toughest obstacles.
As we control every component of hardware design, manufacturing, data processing, and software engineering, our office is a truly inspiring mix of experts from a variety of domains.
We have a people-centric approach toward culture and community and we strive to iterate i

In [4]:
from langchain_core.prompts import PromptTemplate


extract_prompt = PromptTemplate.from_template(
    """### SCRAPED TEXT FROM WEB PAGE:
    {page_data}
    ### TASK:
    Scraped text from a web page is provided. Create a json object with the following fields:
    - role: The role of the job
    - experience: The experience required for the job
    - skills: The skills required for the job
    - description: The description of the job
    Only return the json object without any additional text.
    ### JSON OBJECT (NO PREAMBLE TEXT):
    """)

In [5]:
chain_extract = extract_prompt | llm
result = chain_extract.invoke(input = {"page_data":page_data} )
print(result.content)

{
"role": "Working Student, Image Quality Support",
"experience": "Pursuing a B.S or M.S Degree in an Engineering, Software, or Science related field from an accredited college or technical school",
"skills": ["Fundamentals of remote sensing and image processing", "Good knowledge of photogrammetry and GIS systems", "Familiarity with UNIX operating systems", "Proficiency in English", "Python skills"],
"description": "We are currently seeking a student assistant to support our Operations team. The successful candidate will work in the Image Quality Support (IQS) team. The IQS team is dedicated to comprehensive image quality evaluation, covering all aspects of image integrity."
}


In [6]:
type(result.content)

str

In [7]:
json_res = eval(result.content)

In [8]:
type(json_res)

dict

In [9]:
import pandas as pd
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, "assets")
csv_path = os.path.join(assets_dir, "portfolio.csv")
df = pd.read_csv(csv_path)
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
import chromadb

client = chromadb.PersistentClient(os.path.join(parent_dir,'vector_store'))
collection = client.get_or_create_collection("portfolio")

if collection.count() == 0:
    for index, row in df.iterrows():
        collection.add(
            documents=row["Techstack"],
            metadatas={"links": row["Links"]},
            ids=[str(index)]
        )

In [11]:
links = collection.query(query_texts=["Experence in Python","Experence in Javascript"], n_results=2).get("metadatas")

In [12]:
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}]]

In [13]:
skills = json_res["skills"]

In [14]:
links = collection.query(query_texts=skills, n_results=2).get("metadatas")
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [15]:
email_prompt = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    ### BACKGROUND:
    You are Abhi, a Business Development Executive at DIT Technologies that provides various IT solutions.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering
    scalability, process optimization, cost reduction, and heightened overall efficiency.
    ### TASK: 
    Your job is to write a cold email to the client regarding the job mentioned above describing
    the capability of DIT Technologies in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase DIT's portfolio:
    {link_list}
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
        
    """
)

In [16]:
chain_email = email_prompt | llm
email = chain_email.invoke(input = {"job_description":json_res, "link_list":links})
print(email.content)

Here is the cold email:

Subject: Expert Image Quality Support for Your Operations Team

Dear Hiring Manager,

I came across your job posting for a Working Student, Image Quality Support, and I believe DIT Technologies can provide the ideal candidate to support your Operations team. Our team of experts has extensive experience in remote sensing, image processing, and GIS systems, making us a great fit for your Image Quality Support (IQS) team.

At DIT Technologies, we have a proven track record of delivering tailored solutions that drive scalability, process optimization, cost reduction, and heightened overall efficiency. Our team is well-versed in Python, UNIX operating systems, and photogrammetry, ensuring seamless integration with your existing systems.

To give you an idea of our capabilities, I'd like to highlight some of our relevant projects:

* Our machine learning and Python expertise can be seen in our portfolio: https://example.com/ml-python-portfolio
* We have also successf